# Postprocessing
-------

Run the popstprocessing scripts on the two nearby galaxies with the Dale+(2017) photometry. We have fit these three different ways:

1. MCMC exploration with very long chains
2. MLE with L-BFGS-B algorithm
3. Brief MCMC exploration initialized in a Gaussian ball around the MLE

and so we'll compile three catalogs as a result.

## Imports

In [1]:
import numpy as np
import h5py 
import glob

from lightning.postprocessing import postprocess_catalog

## 1. Pure MCMC

In [2]:
chain_filenames = ['ngc337_chains.hdf5',
                   'ngc628_chains.hdf5']

config_filenames = ['ngc337_config.json',
                    'ngc628_config.json']

postprocess_catalog(chain_filenames,
                    config_filenames,
                    solver_mode='mcmc',
                    model_mode='json',
                    names=None, # it'll figure it out
                    catalog_name='postprocessed_catalog_puremcmc.hdf5')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.42s/it]


In [19]:
f = h5py.File('postprocessed_catalog_puremcmc.hdf5')
print('A sampling of the locations in the file. Note the hierarchical ordering, which is weirdly hard to visualize with the Python HDF5 API.')
print('I have a CLI app called "h5tree" I ripped off from a stackoverflow answer that prints out the hierarchical structure,')
print('which I can provide on request.')
print(f.keys())
print(f['ngc337'].keys())
print(f['ngc337/mcmc'].keys())
print(f['ngc337/parameters'].keys())
print(f['ngc337/parameters/sfh'].keys())
print(f['ngc337/parameters/sfh/psi_1'].keys())
print(f['ngc337/properties'].keys())
f.close()

A sampling of the locations in the file. Note the hierarchical ordering, which is weirdly hard to view with the Python HDF5 API.
I have a CLI app called "h5tree" I ripped off from a stackoverflow answer that prints out the hierarchical structure,
which I can provide on request.
<KeysViewHDF5 ['ngc337', 'ngc628']>
<KeysViewHDF5 ['mcmc', 'parameters', 'properties']>
<KeysViewHDF5 ['logprob_samples', 'samples']>
<KeysViewHDF5 ['atten', 'dust', 'sfh']>
<KeysViewHDF5 ['psi_1', 'psi_2', 'psi_3', 'psi_4', 'psi_5']>
<KeysViewHDF5 ['best', 'hi', 'lo', 'med']>
<KeysViewHDF5 ['filter_labels', 'lnu', 'lnu_unc', 'lumdist', 'mstar', 'pvalue', 'redshift', 'steps_bounds']>


## 2. MLE only 

In [16]:
res_filenames = ['ngc337_mle_res.h5',
                 'ngc628_mle_res.h5']

config_filenames = ['ngc337_config.json',
                    'ngc628_config.json']

postprocess_catalog(res_filenames,
                    config_filenames,
                    solver_mode='mle',
                    model_mode='json',
                    names=None, # it'll figure it out
                    catalog_name='postprocessed_catalog_mle.hdf5')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.14s/it]


In [21]:
f = h5py.File('postprocessed_catalog_mle.hdf5')
print('Note now that when we get all the way down into the parameters, we only have the best fit and not the quantiles.')
print('On its own, the current MLE implementation does not estimate the uncertainties; for taht you need to run the MCMC followup.')
print(f.keys())
print(f['ngc337'].keys())
print(f['ngc337/res'].keys())
print(f['ngc337/parameters'].keys())
print(f['ngc337/parameters/sfh'].keys())
print(f['ngc337/parameters/sfh/psi_1'].keys())
print(f['ngc337/properties'].keys())
f.close()

Note now that when we get all the way down into the parameters, we only have the best fit and not the quantiles.
On its own, the current MLE implementation does not estimate the uncertainties; for taht you need to run the MCMC followup.
<KeysViewHDF5 ['ngc337', 'ngc628']>
<KeysViewHDF5 ['parameters', 'properties', 'res']>
<KeysViewHDF5 ['bestfit', 'chi2_best']>
<KeysViewHDF5 ['atten', 'dust', 'sfh']>
<KeysViewHDF5 ['psi_1', 'psi_2', 'psi_3', 'psi_4', 'psi_5']>
<KeysViewHDF5 ['best']>
<KeysViewHDF5 ['filter_labels', 'lnu', 'lnu_unc', 'lumdist', 'mstar', 'redshift', 'steps_bounds']>


## 3. MLE with MCMC followup

In [22]:
chain_filenames = ['ngc337_mlemcmc_chains.h5',
                   'ngc628_mlemcmc_chains.h5']

config_filenames = ['ngc337_config.json',
                    'ngc628_config.json']

postprocess_catalog(chain_filenames,
                    config_filenames,
                    solver_mode='mcmc',
                    model_mode='json',
                    names=None, # it'll figure it out
                    catalog_name='postprocessed_catalog_mlemcmc.hdf5')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.47s/it]


In [26]:
f = h5py.File('postprocessed_catalog_mlemcmc.hdf5')
print('The structure of this file is exactly the same as the "pure mcmc" file.')
print('You can interpret it exactly the same as we might with a typical MCMC sampling,')
print('reporting the median values of the parameters/quantities of interest (though we ran it with much shorter chains, so beware),')
print('or you could use the inter-quantile range to estimate the 1 sigma uncertainty around the best fit.')
print(f.keys())
print(f['ngc337'].keys())
print(f['ngc337/mcmc'].keys())
print(f['ngc337/parameters'].keys())
print(f['ngc337/parameters/sfh'].keys())
print(f['ngc337/parameters/sfh/psi_1'].keys())
print(f['ngc337/properties'].keys())
f.close()

The structure of this file is exactly the same as the "pure mcmc" file.
You can interpret it exactly the same as we might with a typical MCMC sampling,
reporting the median values of the parameters/quantities of interest (though we ran it with much shorter chains, so beware),
or you could use the inter-quantile range to estimate the 1 sigma uncertainty around the best fit.
<KeysViewHDF5 ['ngc337', 'ngc628']>
<KeysViewHDF5 ['mcmc', 'parameters', 'properties']>
<KeysViewHDF5 ['logprob_samples', 'samples']>
<KeysViewHDF5 ['atten', 'dust', 'sfh']>
<KeysViewHDF5 ['psi_1', 'psi_2', 'psi_3', 'psi_4', 'psi_5']>
<KeysViewHDF5 ['best', 'hi', 'lo', 'med']>
<KeysViewHDF5 ['filter_labels', 'lnu', 'lnu_unc', 'lumdist', 'mstar', 'pvalue', 'redshift', 'steps_bounds']>
